In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# train = pd.read_csv("/kaggle/input/spot-mask/train_labels.csv")

train = pd.read_csv("/kaggle/input/tuberculosis-xray/Train.csv")

test = pd.read_csv("/kaggle/input/tuberculosis-xray/Test.csv")

train["LABEL"] = [str(i) for i in train["LABEL"]]

train["new_filenames"] = [(str(i) + ".png") for i in train["ID"]]

test["new_filenames"] = [(str(i) + ".png") for i in test["ID"]]

train_img, val_img = train_test_split(train, test_size = 0.2)

import os
# all_dir = os.listdir("/kaggle/input/spot-mask/images/images")

# all_dir = os.listdir("/kaggle/input/spot-mask/images/images")

# train_dir = train["image"].tolist()

# test_dir = [item for item in all_dir if item not in train_dir]

# test_img = pd.DataFrame({"image": test_dir})

In [ ]:
img_size = 480


train_datagen = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.2,
                                 height_shift_range=0.2,
                                  zoom_range=0.2,
                                horizontal_flip=True)

train_generator = train_datagen.flow_from_dataframe(
                    dataframe = train_img,
                    directory = "/kaggle/input/tuberculosis-xray/train_small/train_small",
                    
                    x_col = "new_filenames",
                    y_col = "LABEL",
                    target_size = (img_size, img_size),
                    class_mode = "binary"
                    )


val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = train_datagen.flow_from_dataframe(
                    dataframe = val_img,
                    directory = "/kaggle/input/tuberculosis-xray/train_small/train_small",
                    x_col = "new_filenames",
                    y_col = "LABEL",
                    target_size = (img_size, img_size),
                    class_mode = "binary"
                    )


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
                    dataframe = test,
                    directory = "/kaggle/input/tuberculosis-xray/test_small/test_small",
                    x_col = "new_filenames",
                    target_size = (img_size, img_size),
                    class_mode = None)

In [ ]:
#train_generator.class_indices

In [ ]:
num_epochs = 10

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'])


history = model.fit(
      train_generator,
      epochs= num_epochs,
      validation_data = val_generator,
      #callbacks = [es],
      verbose=1)

In [ ]:
from tensorflow.keras.applications import InceptionV3, ResNet50, MobileNetV2


num_epochs = 10

model = tf.keras.models.Sequential([
    InceptionV3(include_top=False, weights='imagenet', input_shape = (img_size, img_size, 3)),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    #tf.keras.layers.Dense(1024, activation='relu'),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')])

model.layers[0].trainable = False

model.compile(loss='binary_crossentropy',
              optimizer= "adam",
              #optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

history = model.fit(
      train_generator,
      epochs=num_epochs,
      validation_data = val_generator,
      verbose=1)

In [ ]:
filename = "sub2.csv"

prediction = model.predict(test_generator)

pd.DataFrame({"ID": test["ID"], "LABEL": prediction.flatten()}).to_csv("/kaggle/working/"+filename, index = False)